In [1]:
!git clone https://github.com/Mjrovai/MachadoAssisBot

Cloning into 'MachadoAssisBot'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 26 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), 1.87 MiB | 4.01 MiB/s, done.


In [2]:
import shutil

shutil.move('./MachadoAssisBot/machado', './')
shutil.rmtree('./MachadoAssisBot')

In [3]:
path = './machado/'

text1 = open(path+'casmurro.txt', 'r').read()
text2 = open(path+'mao_luva.txt', 'r').read()
text3 = open(path+'memorias.txt', 'r').read()
text4 = open(path+'quincas.txt', 'r').read()
text5 = open(path+'esau_jacob.txt', 'r').read()
text6 = open(path+'memorial.txt', 'r').read()
text7 = open(path+'papeis.txt', 'r').read()

text = text1+text2+text3+text4+text5+text6+text7

In [4]:
machado = open('machado.txt','w')
machado.write(text)
machado.close()

In [5]:
text = open('machado.txt', 'r').read()
vocab = sorted(set(text))

In [6]:
import numpy as np

char_to_ind = {char:ind for ind, char in enumerate(vocab)}
ind_to_char = np.array(vocab)
encoded_text = np.array([char_to_ind[c] for c in text])

In [7]:
seq_len = 120
total_num_seq = len(text)//(seq_len+1)

In [8]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [9]:
import tensorflow as tf

char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

dataset = sequences.map(create_seq_targets)

In [10]:
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy

def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [12]:
vocab_size = len(vocab)
embed_dim = 64
rnn_neurons = 1026

In [13]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size
)

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):

  example_batch_predictions = model(input_example_batch)

In [15]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [16]:
epochs = 30

In [17]:
model.fit(dataset,epochs=epochs)
model.save('machado_gen.h5')

Epoch 1/30
156/156 [==============================] - 28s 135ms/step - loss: 2.8601
Epoch 2/30
156/156 [==============================] - 26s 133ms/step - loss: 2.1922
Epoch 3/30
156/156 [==============================] - 24s 139ms/step - loss: 1.9552
Epoch 4/30
156/156 [==============================] - 25s 142ms/step - loss: 1.7478
Epoch 5/30
156/156 [==============================] - 24s 139ms/step - loss: 1.5944
Epoch 6/30
156/156 [==============================] - 24s 143ms/step - loss: 1.4951
Epoch 7/30
156/156 [==============================] - 24s 139ms/step - loss: 1.4261
Epoch 8/30
156/156 [==============================] - 23s 137ms/step - loss: 1.3781
Epoch 9/30
156/156 [==============================] - 24s 140ms/step - loss: 1.3416
Epoch 10/30
156/156 [==============================] - 24s 142ms/step - loss: 1.3125
Epoch 11/30
156/156 [==============================] - 23s 137ms/step - loss: 1.2881
Epoch 12/30
156/156 [==============================] - 24s 138ms/step - lo

In [18]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('machado_gen.h5')
model.build(tf.TensorShape([1, None]))

In [19]:
def generate_text(model, start_seed, gen_size=100, temp=1.0):
    num_generate = gen_size

    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temp

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [20]:
print(generate_text(model,"Maria",gen_size=1000))

Mariadente; de todo caso, porém, eu estou
frentes a um peralta noite.

Ita vez
philosophando triste o conhe, despeu e vinha á vontade do joelho, e uma parenta do arciber
fizera vivos e calvais amigos.

O Alistia dos gemeos são declarou a
impressão que Flora era como se ouvessa, Santos, em gostamento, e acho Ayres
fechal-o, mandava vinte e dous, e lá se ia a mim. Emfim, chegou-se á mema; elle, absolvado das
respeitamente que calarres da obra demasiada forte e de pae depressa, logo, sem que
desejasse de bom humano. Cheguei a vista.
Assim fôra tevia a condição do ordeno adoecer: assim á janella, onde não achasse ninguem; tinha os reunte da
hyputação tocava nos seus chefes politicos desembargadores, tanto fazendo que lêm e haveria de bruços e pena, esmer que
ella faz uma só cessaria nunca; desejava, apazece antes o nome proposta, deixava-se isto
e alegre, sem que outo posso effeito, segundo as ruas estava grave, Mas os doentes furtos e os
primeiros frequentes, e foi convidando-me em o seu 

In [21]:
print(generate_text(model," CASMURRO ",gen_size=1000))

 CASMURRO CLXXV

Ora foi por ora;
minha impressão se seria perdel-os; eu acadeiramente absurda, e bem a
menina; mas eu á portholar tão expansido, que o não acontecea. Na
acção do ultimo encerroguro da casa; ouviu Braz Cubas_, olhou para fóra, a
ponta do nariz, calado, uma vontade de cobrada, ao contrario: os alejantes de inveja era sempendi ella ou e
ver se podem vivia de uma calumnia, até que
dormia, se este
e aos seus proprios sentidos. Desappertava-se muccumbrança e fue
talvez não poderia nda, e beaças. D. Rita ficou sem conhecer para
todas as sobrancelhas, faziam o sentido
de cabellos, e me disse que era boa. A moça não é
gastadeiras alegres: os bensfitou da cama Alhas e bebdes... passaram planos durante alguns minutos.
Examinhucei pendurado, com o dedoculo, por um bemso
parecido com as mesmascasta e sair; queria doente.

--Mrs. Oswald, disse-lhe que o Livro é destinar a nté
que consultassem. Carlos Maria; ainda meteda para saber se assim
explicado, não entendia cá para as leis. Qu

In [22]:
print(generate_text(model," Rio de Janeiro  ",gen_size=5000))

 Rio de Janeiro  catas de anterior;
onde fui fazer outra visita podia ter.




CXXXVII

A mãe estavam alegre.
Depois lhe queira, vi jantar commigo. Simão
Bacamarte não foi claro ou todo o embarcar. Já extrahos na qualidade
dos homens. Tambem não fazia desejar,--ou que fim rarões, o
fechor da acção ao Palha; mas não creia de uma casa visinha.

--Sahiu, vou genio e cedo-se a tro estante. Vamos
resistir-lhe a mesma cousa rice, dez minutos, que o Aguiar
entrou a falar
delle. Estevão era feio que tambem foi liberal, sinhá
fel-a demorar as almas. Paulo houve que Lopo Alves alem de lá conferente á casa
della por tuas vozes, senão acceital, nem a occasíão tive de deu um para outro
nome, e por outras tantas tiveras jantas. Da nossa todos os dias explicaram logo, e pedia a ponta de
um pae, lentamente para elle jorá a ir comprimental-o.

Rubião cada pertão e resolutra contralidade de D. Maria
da Gloria (em Voss vines, e.... Inventadas cerradas ou só horas. D. Tonica, que quando tornava
aos pés de

In [ ]:
print(generate_text(model,"A LUVA DE CASMURRO ",gen_size=5000))

In [ ]:
print(generate_text(model,"A LUVA DE CASMURRO ",gen_size=5000, temp=0.5))
